# Workshop Notebook

This jupyter notebook will make for the interactive part of this workshop

## Step 1: Inspect the data

Usually, the first thing we want to do when dealing with any new type of data, we want to inspect it first to get some intuitions for it. By visualizing the data, we can often get some ideas as to how to tackle the data and features we can extract from it.

In [1]:
import pandas as pd

label_file = 'data/training_labels.csv'
df = pd.read_csv(label_file)
df


What has been done here, is to load a csv file containing rows of filepaths and correspendong train types. The filepaths are stored as binary blobs which can be found in data/signals. The table shown above is an excerpt of this list as it has been read into a dataframe
Let us explore a couple of the signatures we can find there. I also encourage you to look at more of them to get an even better idea of the data


In [2]:
import matplotlib.pyplot as plt
from helpers import load_binary
from helpers import plot_size
%matplotlib inline

type_a = df.loc[df['train_type'] == 'train_a']
type_b = df.loc[df['train_type'] == 'train_b']
type_c = df.loc[df['train_type'] == 'train_c']
type_d = df.loc[df['train_type'] == 'train_d']

file_a = 'data/signals/training/' + type_a['filename'].iloc[0]
file_b = 'data/signals/training/' + type_b['filename'].iloc[0]
file_c = 'data/signals/training/' + type_c['filename'].iloc[0]
file_d = 'data/signals/training/' + type_d['filename'].iloc[0]

signal_a = load_binary(file_a)
signal_b = load_binary(file_b)
signal_c = load_binary(file_c)
signal_d = load_binary(file_d)

plot_size(16, 8)
plt.subplot(411)
plt.title('Train A')
plt.plot(signal_a)
plt.subplot(412)
plt.title('Train B')
plt.plot(signal_b)
plt.subplot(413)
plt.title('Train C')
plt.plot(signal_c)
plt.subplot(414)
plt.title('Train D')
plt.plot(signal_d)
plt.tight_layout()
plt.show()


Here we can already see a couple of things to expect from the data. The data comes in timeseries with thousands of timesteps and come with variable lengths. It's filled with impulses, likely from when wheels of the train is passing over the sensor. This means a couples of things already:
1. The data will not go well with a lot of Machine Learning methods as is, as there are too many timesteps in the signal
2. When trying to fit a model, we would want to consider some way of making the signals the same size

To get a better idea of the ranges of the data, we ought to compute some aggregate statistics as well

In [4]:
import numpy as np

# Allocate a bit of space for storing data
lengths = []
rms_values = []
value_max = []

# Iterate over all the signals and collect data from them
for i in range(len(df)):
    filename = df['filename'][i]
    train_type = df['filename'][i]
    
    # uncomment the next 2 lines to filter for specific train types
    # if train_type != 'train_a':
    #     continue
    
    data = load_binary('data/signals/training/' + filename)
    rms = np.sqrt(np.mean(np.square(data)))  # Compute the Root Mean Square
    length = len(data)
    
    # Append values to lists
    lengths.append(length)
    rms_values.append(rms)
    value_max.append(max(np.abs(data)))

# Plot histograms and a bar plot of the gathered data to determine the spread of values
plot_size(16, 8)
plt.subplot(141)
plt.title('Signal Lengths')
plt.hist(lengths)
plt.subplot(142)
plt.title('Signal RMS')
plt.hist(rms_values)
plt.subplot(143)
plt.title('Signal Maximas')
plt.hist(value_max)
plt.subplot(144)
plt.title('Train Type Distribution')
df['train_type'].value_counts().plot(kind='bar')
plt.tight_layout()
plt.show()


From this, we can see that most of the signals are around 20000 timesteps long or shorter. We also see that there is quite a range of both maximum values of RMS values, but with stronger tendencies towards the lower end of the spectrum. We also see that we have a relatively even spread of train types, and also a category called "unknown". These are signals for which the train type is not known. The reason we look at this is to determine a number of things
1. See if there are strong tendencies in the data for which we can make simple filters
2. To help determine how data needs to be transformed for a potential model
3. Determine if special weighting or other techniques are required to deal with imbalanced datasets

Questions:
- What would you look at next with basis in this data?
- What is the simplest model you could use as a baseline for determining the train type here?


## Step 2: Simple approaches to modelling the data

Let's start with a couple of simple ways to model the data.
There's a couple of ways we can go about it.
- We can try to extract simple features from the signal to work with, and apply it to a modelling technique of our choice
- We can try to use the signals themselves as features within a modelling architecture

For simplicity, and time reasons we'll be building Neural Networks using a library called "Keras". If you're familiar with scikit-learn, feel free to use it to model data as well. Let's start with a very simple model using the 3 features from before


In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from helpers import train_to_id5
from helpers import load_dataset


# A function to extract the values we need as input and output for the model training
# Note: You can make changes here to look at different features
def extract_features(signals, train_types):
    model_input = []
    model_target = []
    
    # Iterate over all signals and corresponding train types
    for signal, train_type in zip(signals, train_types):
        
        # Extract signal features
        rms = np.sqrt(np.mean(np.square(signal)))
        max_value = np.max(np.abs(signal))
        length = len(signal) / 1000.0  # Lower this value to a similar output as the other features
        
        # Assemble these values into a single data point / array
        input_vector = [rms, max_value, length]
    
        # Convert train type to number
        target = train_to_id5(train_type)
        
        # Add to dataset to be fed to a machine learning algorithm
        model_input.append(input_vector)
        model_target.append(target)
    
    # Convert to a more digestable format and return the data
    model_input = np.array(model_input)
    model_target = np.array(model_target)
    return model_input, model_target


# Load the data
training_x, training_y = load_dataset(dataset='training')
validate_x, validate_y = load_dataset(dataset='validate')

# Transform the data / extract features
training_x, training_y = extract_features(training_x, training_y)
validate_x, validate_y = extract_features(validate_x, validate_y)

# Build a simple Neural Network
model = Sequential()
model.add(Dense(units=5, input_dim=training_x.shape[1]))
model.add(Dense(units=10))
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Apply the data and the train types and have the algorithm fit a model from x to y
# Quick primer on fitting:
# epochs = number of iterations spent trying to fit the model output to the target output
# validation_data = data not used for fitting the model. Used to determine how well the model works on unseen data
# batch_size = number of samples to update over in one step of the optimizing algorithm.
#   with batch_size, usually more means faster fitting time and more stable results, but worse generalization
logger = model.fit(training_x, training_y, epochs=250, batch_size=32, validation_data=[validate_x, validate_y])

# Visualize the fitting process to learn about how good the model likely is
# Note, validation accuracy is the metric of how good the model probably is, while training accuracy shows
# how quickly the algorithm found a way to map the input to the desired output
plt.title('Accuracy over epochs')
plt.plot(logger.history['accuracy'])
plt.plot(logger.history['val_accuracy'])
plt.legend(['training accuracy', 'validation accuracy'])
plt.xlabel('Epochs')
plt.tight_layout()
plt.show()


Okay, we have improved on the baseline by a bit, but can we do better? Let us try to extract features from the signal directly. Note that in order to do so, however, we need to work with the signal a little bit first


In [ ]:
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from helpers import train_to_id5


# Change this to set how many steps long you want your time-series to be
input_length = 10000


# A function to extract the values we need as input and output for the model training
# Note: You can make changes here to look at different features
def extract_features(signals, train_types):
    model_input = []
    model_target = []
    
    # Iterate over all signals and corresponding train types
    for signal, train_type in zip(signals, train_types):
                
        # Assemble the signal one data point
        input_vector = np.reshape(signal, (-1, 1))  # special case if you have only 1 time series
        # input_vector = [signal, signal]  # uncomment to add multiple equally long time series to your model input
    
        # Convert train type to number
        target = train_to_id5(train_type)
        
        # Add to dataset to be fed to a machine learning algorithm
        model_input.append(input_vector)
        model_target.append(target)
    
    # Convert to a more digestable format and return the data, also makes also signals equally long
    model_input = sequence.pad_sequences(model_input, input_length)
    model_target = np.array(model_target)
    return model_input, model_target


# Load the data
training_x, training_y = load_dataset(dataset='training')
validate_x, validate_y = load_dataset(dataset='validate')

# Transform the data / extract features
training_x, training_y = extract_features(training_x, training_y)
validate_x, validate_y = extract_features(validate_x, validate_y)


# Build a Convolutional Neural Network
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=5, padding='valid', input_shape=training_x.shape[1:]))
model.add(MaxPool1D(2))
model.add(Conv1D(filters=8, kernel_size=5, padding='valid'))
model.add(MaxPool1D(2))
model.add(Flatten())
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


# Fit a model to the data. Note less epochs are needed here
logger = model.fit(training_x, training_y, epochs=25, batch_size=32, validation_data=[validate_x, validate_y])

# Visualize the fitting process to learn about how the model likely is
plt.title('Accuracy over epochs')
plt.plot(logger.history['accuracy'])
plt.plot(logger.history['val_accuracy'])
plt.legend(['training accuracy', 'validation accuracy'])
plt.xlabel('Epochs')
plt.tight_layout()
plt.show()

Now it is your turn. How good a model can you build for determining the correct train types? Some things to explore
- Can you extract some better features from the signal to build a better model?
- Does tweaking with the model parameters help the output? (change the amount of units, the amount of layers, etc.)
- What is the most important qualities for the output of the model? Are there better ways to achieve this?
- Maybe combining multiple streams of data gives better results

In the "examples" notebook you will find code to help you do different things suggested. You can copy paste them into the snippets above or below, or you could also make an entirely new code segment combining different things. Your goal is to try to achieve as high stable results on the validation data as possible. To do this, you will need to run the model a couple of times since there is a bit of randomness in these models which lead to different results each time.
Try to not use too high numbers in the model parameters however, as the training time will end up being far too long for this session if you do.
Good luck!


In [ ]:
# Feel free to paste code into here if you want to keep the code above clean and copy-pastable
